In [37]:
import numpy as np
import time
from PIL import Image
import os
import sys
from numba import cuda

In [17]:
def generate_hooks(n_hooks, wheel_pixel_size):
    
    r = (wheel_pixel_size / 2) - 1
    
    theta = np.arange(n_hooks, dtype="float64") / n_hooks * (2 * np.pi)
    
    x = r * (1 + np.cos(theta)) + 0.5
    y = r * (1 + np.sin(theta)) + 0.5
    
    return np.array((x,y)).T


def through_pixels(p0, p1):
    
    d = max(int(((p0[0]-p1[0])**2 + (p0[1]-p1[1])**2) ** 0.5), 1)
    
    pixels = p0 + (p1-p0) * np.array([np.arange(d+1), np.arange(d+1)]).T / d
    pixels = np.unique(np.round(pixels), axis=0).astype(int)
    
    return pixels


def build_through_pixels_dict(n_hooks):

    n_hook_sides = n_hooks

    l = [(0,1)]
    for j in range(n_hook_sides):
        for i in range(j):
            l.append((i, j))
    
    random_order = np.random.choice(len(l),len(l),replace=False)
    
    d = {}    
    t_list = []
    t0 = time.time()
    
    for n in range(len(l)):
        (i, j) = l[random_order[n]]
        p0, p1 = hooks[i], hooks[j]
        d[(i,j)] = through_pixels(p0, p1)
        
        t = time.time() - t0
        t_left = t * (len(l) - n - 1) / (n + 1)
        print(f"time left = {time.strftime('%M:%S', time.gmtime(t_left))}", end="\r")
    
    return d

In [26]:
n_hooks = 20
wheel_pixel_size = 500
hook_pixel_size = 0
hooks = generate_hooks(n_hooks, wheel_pixel_size)
through_pixels_dict = build_through_pixels_dict(n_hooks)

In [28]:
test = np.zeros((500, 500))
print(test.shape)
count = 0
for pixels in through_pixels_dict.values():
    for pixel in pixels:
        count += 1
        test[pixel[0], pixel[1]] = 255
print(count)
image = Image.fromarray(test)
image.show()


(500, 500)
59393


In [36]:
cwd = os.getcwd()
image = Image.open("audrey.jpg").convert("L")
# Convert the image to a NumPy array
image_array = np.array(image)

# Define the center and radius of the circle
cx, cy = 250, 250
r = 250

# Create a grid of x and y coordinates
x, y = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

# Compute the distance from each pixel to the center of the circle
distances = np.sqrt((x - cx)**2 + (y - cy)**2)

# Create a boolean mask that is True for pixels inside the circle
mask = distances < r

# Filter the pixel values using the mask
image_array[~mask] = 0

# Convert the NumPy array back to an image
new_image = Image.fromarray(image_array)

# Display the new image
new_image.show()

print(image_array[0,0])

0


In [50]:
def compute_norm(through_pixels_dict): 
    cur_min, (target_i, target_j) = sys.maxsize, (0, 0)
    for (i, j), pixels in through_pixels_dict.items():
        # Compute test_line and ref using array indexing
        test_line = building[pixels[:, 0], pixels[:, 1]] + 10
        ref = image_array[pixels[:, 0], pixels[:, 1]]

        # Clip test_line between 0 and 255 and compute the norm
        arr_test_line = np.clip(test_line, a_min=0, a_max=255)
        arr_ref = np.array(ref)
        norm = np.linalg.norm(arr_test_line - arr_ref)
        
        # Update cur_min and (target_i, target_j) if necessary
        if norm < cur_min:
            cur_min = norm
            (target_i, target_j) = (i, j)

    # Update the building array and clip between 0 and 255
    pixels = through_pixels_dict[(target_i, target_j)]
    building[pixels[:, 0], pixels[:, 1]] += 10
    np.clip(building, a_min=0, a_max=255)
    return cur_min, (target_i, target_j)



In [55]:
building = np.zeros((500, 500))

prev_min, (i, j) = compute_norm(through_pixels_dict)
del through_pixels_dict[(i, j)]

cur_min, (i, j) = compute_norm(through_pixels_dict)
del through_pixels_dict[(i, j)]

max_iters = 100000
t0 = time.time()
for num_iters in range(max_iters):
    # if cur_min >= prev_min:
    #     # break
    prev_min = cur_min
    cur_min, = compute_norm(through_pixels_dict)[:1]

    # t = time.time() - t0
    # t_left = t * (max_iters) / (num_iters + 1)
    # print(f"time left = {time.strftime('%M:%S', time.gmtime(t_left))}", end="\r")


image = Image.fromarray(building)
image.show()